In [1]:
#importing the relevant libraries
import pandas as pd
import numpy as np
import os
from glob import glob
import sys
pd.set_option('display.max_columns', None, 'display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# pd.set_option('max_colwidth',-1)

In [13]:
dta = pd.read_csv('./sampledata/Test/Bejing/Race/Analysis_Race.CSV', sep=";", encoding = "ISO-8859-1")
dtc = pd.read_csv('./sampledata/Test/Bejing/Race/Classification_Race.CSV', sep=";", encoding = "ISO-8859-1")
dtc['DRIVER_NAME'] = dtc['DRIVER_FIRSTNAME'] + " " + dtc['DRIVER_SECONDNAME']
new_test = dta.merge(dtc, on="DRIVER_NAME")
new_test = new_test.rename(columns=lambda x: x.strip())

In [14]:
new_test = new_test.drop(columns = ['DRIVER_NUMBER', 'LAPS', 'FL_KPH', 'HOUR', 'TOTAL_TIME',
       'NUMBER', 'DRIVER_SHORTNAME', 'S2_LARGE',
       'FL_LAPNUM', 'VEHICLE',
       'LAP_IMPROVEMENT', 'CLASS', 'S2_IMPROVEMENT', 'TOP_SPEED',
       'DRIVER_COUNTRY', 'STATUS', 'ELAPSED', 'ï»¿POSITION', 'TIRES',
       'S3_LARGE', 'GAP_PREVIOUS', 'ï»¿NUMBER',
       'GAP_FIRST', 'DIVISION', 'DRIVER_SECONDNAME', 'DRIVER_FIRSTNAME',
       'LAP_TIME', 'S1_IMPROVEMENT', 'DRIVER_HOMETOWN', 'FL_TIME',
       'S1_LARGE', 'DRIVER_LICENSE', 'S3_IMPROVEMENT', 'Unnamed: 21', 'Unnamed: 22'])
len(new_test.keys())

10

In [18]:
new_test['location'] = 'Bejing'
new_test['match_type'] = 'Race'

In [20]:
len(new_test.keys())

12

In [21]:
import math

for i in (['S1', 'S2', 'S3', 'PIT_TIME']):
    p = new_test[i]
    for count, j in enumerate(p):
        x = len(str(j).split(':'))
        if (pd.isna(j)):
            p[count] = "00:00:00"
        elif (x == 1):
            p[count] = "00:00:" + str(j)
        elif (x == 2):
            p[count] = "00:" + str(j)
        elif (x == 3):
            p[count] = "" + str(j)
        p[count] = float(p[count].split(":")[0])*3600000 + float(p[count].split(":")[1])*60000 + float(p[count].split(":")[2])*1000
    new_test[i] = p
    
new_test[['KPH', 'S1', 'S2', 'S3', 'LAP_NUMBER', 'PIT_TIME']] = new_test[['KPH', 'S1', 'S2', 'S3', 'LAP_NUMBER', 'PIT_TIME']].apply(pd.to_numeric)

<ipython-input-21-c6a8297bc89c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p[count] = "00:" + str(j)
<ipython-input-21-c6a8297bc89c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p[count] = float(p[count].split(":")[0])*3600000 + float(p[count].split(":")[1])*60000 + float(p[count].split(":")[2])*1000
<ipython-input-21-c6a8297bc89c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p[count] = "00:00:" + str(j)
<ipython-input

In [22]:
new_test.head(5)

,LAP_NUMBER,CROSSING_FINISH_LINE_IN_PIT,S1,S2,S3,KPH,DRIVER_NAME,PIT_TIME,TEAM,GROUP,location,match_type
0,1,NaN,2295917.0,37598.0,36688.0,5.2,Jarno Trulli,2259620.0,Trulli Formula E Team,1,Bejing,Race
1,2,NaN,39052.0,35808.0,37736.0,110.4,Jarno Trulli,0.0,Trulli Formula E Team,1,Bejing,Race
2,1,NaN,42306.0,36207.0,35301.0,109.2,Lucas di Grassi,0.0,Audi Sport ABT Formula E Team,3,Bejing,Race
3,2,NaN,39864.0,73195.0,63147.0,70.5,Lucas di Grassi,0.0,Audi Sport ABT Formula E Team,3,Bejing,Race
4,3,NaN,92821.0,96555.0,85762.0,45.2,Lucas di Grassi,0.0,Audi Sport ABT Formula E Team,3,Bejing,Race


In [26]:
new_test[['GROUP']] = new_test[['GROUP']].fillna(0)
new_test[['CROSSING_FINISH_LINE_IN_PIT']] = new_test[['CROSSING_FINISH_LINE_IN_PIT']].fillna('')
# new_test['GROUP'] = new_test['GROUP'].replace({'A':0,'B':0})

In [24]:
new_test.head(3)

,LAP_NUMBER,CROSSING_FINISH_LINE_IN_PIT,S1,S2,S3,KPH,DRIVER_NAME,PIT_TIME,TEAM,GROUP,location,match_type
0,1,,2295917.0,37598.0,36688.0,5.2,Jarno Trulli,2259620.0,Trulli Formula E Team,1,Bejing,Race
1,2,,39052.0,35808.0,37736.0,110.4,Jarno Trulli,0.0,Trulli Formula E Team,1,Bejing,Race
2,1,,42306.0,36207.0,35301.0,109.2,Lucas di Grassi,0.0,Audi Sport ABT Formula E Team,3,Bejing,Race


In [28]:
set(new_test['DRIVER_NAME']), max(new_test['LAP_NUMBER'])

({'Charles Pic',
  'Daniel Abt',
  'Franck Montagny',
  'Ho-Pin Tung',
  'Jaime Alguersuari',
  'Jarno Trulli',
  "JÃ©rome d'AmbrosioÂ\xa0",
  'Karun Chandhok',
  'Katherine Legge',
  'Lucas di Grassi',
  'Michela Cerruti',
  'Nelson Piquet JR',
  'Nick Heidfeld',
  'Nicolas Prost',
  'Oriol Servia',
  'Sam Bird',
  'Sebastien Buemi',
  'Stephane Sarrazin',
  'Takuma Sato'},
 25)

In [29]:
Drivers = set(new_test['DRIVER_NAME'])
Max_Laps = max(new_test['LAP_NUMBER'])

In [ ]:
for j in range(1,Max_Laps):
    for i in Drivers:
        